In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import set_config
import BATCHACTIVE
import QlikSense_OEE
import ORDER
import CNTR

# call set_config
dir_sanofi_share = set_config.ConfigSectionMap("SectionOne")['sanofi']
dir_local = set_config.ConfigSectionMap("SectionOne")['local']

# read IP21 SPEED files for AL6 packaging machines
folder = 'IP21_data'

Files = []
df = []



This produces a final dataframe with the following columns

| # | Column | Dtype | Description |  
|---| ------ | ----- | ----------- |  
| 0 |Name|object| 
| 1 |IP_TREND_VALUE|int64||   
| 2 |IP_TREND_QLEVEL|object||  
| 3 |IP_TREND_QSTATUS|object||  
| 4 |Machine|object||  
| 5 |Type|object||  
| 6 |counter_diff|float64|difference between counter files of same name| 
| 7 |time_diff|float64|difference in time between counter files of same name| 
| 8 |In_Changeover|float64|1 if during changeover period (Florian formula), NaN if outside period| 
| 9 |Assembly_Order_No|object| 36630901_ORDERNUMBER file values| 
| 10|PAL_Order_No|object|36650901_ORDERNUMBER file values|  
| 11|BATCH_ID|object|36630901_BATCHID values|  
| 12|BATCH_SIZE|float64|36630901_MATNO values, believe is the number of pens per batch| 
| 13|Shift|category|EARLY, LATE, NIGHT, 06:00 - 14:00, 14:00-22:00, 22:00-06:00|

## FUNCTIONS

In [2]:
def create_df_from_file(indir, infolder, infilesearch):
    folder = infolder
    path = os.path.join(indir, folder)


    df = []
    df_created = pd.DataFrame()

    # do a recursive search for the files now we have multiple dirs
    for filename in iglob(path + '/**/*' + infilesearch + '*', recursive=True):
        df = pd.read_csv(filename)
        df_created = df_created.append(df)

    return df_created


def create_shift_category(passed_df):

    df = []
    df = passed_df
    
    bins = [0,5,13,21,24]


    labels = ['NIGHT','EARLY','LATE','NIGHT']

    df['Shift'] = pd.cut(df.index.hour,bins=bins, include_lowest=True, labels=labels, ordered=False)

    return df


# CHANGEOVER

In [3]:
# call changeover - calculated from the BATCHACTIVE files
df_changeover = BATCHACTIVE.changeover()

# ORDER

In [4]:
# get the batch info 
df_order = ORDER.ORDER_files()

# COUNTER

In [5]:
df_CNTR = CNTR.CNTR_Files()


C:\Users\mark_\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\mark_\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Merge Dataframes

## Create changeover category

In [6]:
df_CNTR.set_index('IP_TREND_TIME', inplace=True)

In [7]:
# create a category of In_Changeover if times are between changeover start and end
for i, times in df_changeover.iterrows():
    df_CNTR.loc[(df_CNTR.index > times[0]) & (df_CNTR.index < times[1]), 'In_Changeover'] = 1

In [23]:
df_CNTR

,Name,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS,Machine,Type,counter_diff,time_diff,In_Changeover
IP_TREND_TIME,,,,,,,,,
2021-08-01 00:07:23.300,36630901_CNTRB,304,Good,Good,Assembly,Bad,NaN,NaN,NaN
2021-08-01 00:17:23.100,36630901_CNTRB,304,Good,Good,Assembly,Bad,0.0,599.8,NaN
2021-08-01 00:27:23.000,36630901_CNTRB,304,Good,Good,Assembly,Bad,0.0,599.9,NaN
2021-08-01 00:32:46.300,36630901_CNTRB,305,Good,Good,Assembly,Bad,1.0,323.3,NaN
2021-08-01 00:32:53.400,36630901_CNTRB,306,Good,Good,Assembly,Bad,1.0,7.1,NaN
...,...,...,...,...,...,...,...,...,...
2021-09-30 23:16:59.600,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.1,NaN
2021-09-30 23:26:59.300,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.7,NaN
2021-09-30 23:36:58.400,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.1,NaN


## Create Order Number column

Find rows for 36630901 order number changes and merge with df_CNTR

In [54]:
df_order.set_index('IP_TREND_TIME', inplace=True)

In [63]:
assembly_orders = df_order[df_order['36630901_ORDERNUMBER'].isna()==False]
pal_orders = df_order[df_order['36650901_ORDERNUMBER'].isna()==False]
# pd.DataFrame({'start':df_orders2.iloc[::2].values, 'end':df_orders2.iloc[1::2].values, 'time_diff_mins':df_orders2.time_diff_mins.iloc[1::2].values})
df_order2 = pd.merge_asof(df_CNTR.sort_index(), df_order.sort_index()[['36630901_ORDERNUMBER','36650901_ORDERNUMBER','BATCH_ID','BATCH_SIZE']].sort_index(), left_index=True, right_index=True, direction='forward', allow_exact_matches=True)
df_order2.rename(columns={'36630901_ORDERNUMBER':'Assembly_Order_No',
                          '36650901_ORDERNUMBER':'PAL_Order_No'}, inplace=True)


,Name,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS,Machine,Type,counter_diff,time_diff,In_Changeover,Assembly_Order_No,PAL_Order_No,BATCH_ID,BATCH_SIZE
IP_TREND_TIME,,,,,,,,,,,,,
2021-08-01 00:02:30.500,36650901_CNTR_BAD_FS_F2,30,Good,Good,Packaging,Bad,NaN,NaN,NaN,80760843.0,NaN,1F111,533064.0
2021-08-01 00:03:22.800,36640901_FAULT,0,Good,Good,Labeler,Bad,NaN,NaN,NaN,80760843.0,NaN,1F111,533064.0
2021-08-01 00:04:08.100,36670901_FAULT,0,Good,Good,Packaging2,Bad,NaN,NaN,NaN,80760843.0,NaN,1F111,533064.0
2021-08-01 00:06:25.800,36680902_FAULT,0,Good,Good,Palletizer,Bad,NaN,NaN,NaN,80760843.0,NaN,1F111,533064.0
2021-08-01 00:06:42.200,36670901_CNTR_GOOD,29926,Good,Good,Packaging2,Good,NaN,NaN,NaN,80760843.0,NaN,1F111,533064.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 23:59:57.200,36640901_CNTRG,100756,Good,Good,Labeler,Good,3.0,1.0,NaN,NaN,NaN,NaN,NaN
2021-09-30 23:59:58.200,36630901_CNTR_GOOD,102227,Good,Good,Assembly,Good,8.0,3.1,NaN,NaN,NaN,NaN,NaN
2021-09-30 23:59:58.200,36640901_CNTRG,100761,Good,Good,Labeler,Good,5.0,1.0,NaN,NaN,NaN,NaN,NaN


## Create Shift Category

In [64]:
# call create_shift_category with dataframe and which column you want to use
df_order2 = create_shift_category(df_order2)

In [66]:
filename = 'Batch_Order_Info.csv'
df_order2.to_csv(os.path.join(dir_sanofi_share, folder, filename))
